# Эксперимент №1. Датасет Титаник

## Предобработка датасета

Предобработка датасета описана в файле prepocess.ipynb

## Загрузка датасета и поиск закономерностей

In [1]:
#!ls
import sys
sys.path.extend(['/home/vladislav/Study/Science work/System'])
import pandas as pd # работа с датасетом
from probconcepts.network.ext_model import ExtBaseModel
from probconcepts.network.process import *
from tqdm import tqdm
from probconcepts.network.ext_mesure import *
from sklearn.metrics import roc_auc_score, precision_score, accuracy_score, recall_score, f1_score
%load_ext autoreload
%autoreload 2

In [2]:
PATH = "exp_titanic/"
ENC_PATH = "exp_titanic/cd_encoings/"
LOGS_PATH = "exp_titanic/logs/"

In [3]:
# загрузка датасета и вывод информации
df = pd.read_csv(PATH + "processed data/train_titanic.csv")
df

,Pclass,Sex,Embarked,Title,Age_cat,Fare_cat,FSize
0,3,male,S,Mr,Student,quart_1,2
1,1,female,C,Mrs,Adult,quart_4,2
2,3,female,S,Miss,Young,quart_1,1
3,1,female,S,Mrs,Young,quart_4,2
4,3,male,S,Mr,Young,quart_2,1
...,...,...,...,...,...,...,...
886,2,male,S,Officer,Young,quart_2,1
887,1,female,S,Miss,Student,quart_3,1
888,3,female,S,Miss,NaN,quart_3,4
889,1,male,C,Mr,Young,quart_3,1


In [4]:
print(df.head().to_latex())

\begin{tabular}{lrlllllr}
\toprule
{} &  Pclass &     Sex & Embarked & Title &  Age\_cat & Fare\_cat &  FSize \\
\midrule
0 &       3 &    male &        S &    Mr &  Student &  quart\_1 &      2 \\
1 &       1 &  female &        C &   Mrs &    Adult &  quart\_4 &      2 \\
2 &       3 &  female &        S &  Miss &    Young &  quart\_1 &      1 \\
3 &       1 &  female &        S &   Mrs &    Young &  quart\_4 &      2 \\
4 &       3 &    male &        S &    Mr &    Young &  quart\_2 &      1 \\
\bottomrule
\end{tabular}



In [8]:
print(df.head().to_latex())

\begin{tabular}{lrlllllr}
\toprule
{} &  Pclass &     Sex & Embarked & Title &  Age\_cat & Fare\_cat &  FSize \\
\midrule
0 &       3 &    male &        S &    Mr &  Student &  quart\_1 &      2 \\
1 &       1 &  female &        C &   Mrs &    Adult &  quart\_4 &      2 \\
2 &       3 &  female &        S &  Miss &    Young &  quart\_1 &      1 \\
3 &       1 &  female &        S &   Mrs &    Young &  quart\_4 &      2 \\
4 &       3 &    male &        S &    Mr &    Young &  quart\_2 &      1 \\
\bottomrule
\end{tabular}



In [8]:
# вывод уникальных значений для каждого признака
i = 0
for col in df:
    print(i, col, df[col].unique())
    i+=1
print("-------------------------------------")
df.info()

0 Pclass [3 1 2]
1 Sex ['male' 'female']
2 Embarked ['S' 'C' 'Q' nan]
3 Title ['Mr' 'Mrs' 'Miss' 'Master' 'Royalty' 'Officer' nan]
4 Age_cat ['Student' 'Adult' 'Young' nan 'babies' 'Teen' 'Children' 'Senior']
5 Fare_cat ['quart_1' 'quart_4' 'quart_2' 'quart_3' nan]
6 FSize [ 2  1  5  3  7  6  4  8 11]
-------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Pclass    891 non-null    int64 
 1   Sex       891 non-null    object
 2   Embarked  889 non-null    object
 3   Title     890 non-null    object
 4   Age_cat   714 non-null    object
 5   Fare_cat  876 non-null    object
 6   FSize     891 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 48.9+ KB


In [9]:
from probconcepts.alg.data import *          # работа с выборкой
from probconcepts.lang.regularity import *   # алгоритм генерации правил
from probconcepts.alg.model import BaseModel 
from probconcepts.alg.generator import *

In [53]:
# генерация выборки (преобразования признаков, создание cd, pe, pt)
cat = [0,2,3,4,5,6] # категориальные признаки
bools = [1]         # булевы признаки

mainSample = Sample(data=df, cat_features=cat, bool_features=bools, 
                cd_output_path=ENC_PATH+"train_cd.json", encoding_output_path=ENC_PATH+"train_encode.json")
train_lbls = pd.read_csv(PATH+"processed data/train_lbls_titanic.csv")
y_train = train_lbls['Survived'].values.flatten()


# генерация тестовой выборки
test = pd.read_csv(PATH + "processed data/test_titanic.csv")
test_lbls = pd.read_csv(PATH + "processed data/submission.csv")
y_test = test_lbls['Survived'].values.flatten()

mainSampleTest = Sample(data=test, cat_features=cat, bool_features=bools, 
                cd_output_path=ENC_PATH+"test_cd.json", encoding_output_path=ENC_PATH+"test_encode.json")

In [11]:
#     Генерация правил №1
# глубина базового перебора -- 1
# ограничение на длину правил -- 150
# минимальный уровень значимости Фишера -- 0.05
# минимальный порог увеличения вероятности правила - 0.0 

md1 = BaseModel(mainSample, base_depth=1, fully_depth=150, confidence_level = 0.05, negative_threshold=0.00,
               rules_write_path = 'exp_titanic/rules/base1/')

# Алгоритм поиска закономерностей: 
# итерация по всем позитивным предикатам в pt
# алгоритм принимает на вход предикат и генерирует все правила, предсказывающие этот предикат,
# а также все правила, предсказывающие отрицание этого предиката

# Поиск правил занял ~40 минут на 1CPU

# для генерации правил раскомментировать следующий код:
"""
for key in mainSample.pt.table.keys():
    for pr in range(len(mainSample.pt.table[key])):
        build_spcr([mainSample.pt.table[key][pr][0]], md1)
        build_spcr([mainSample.pt.table[key][pr][1]], md1)
"""

'\nfor key in mainSample.pt.table.keys():\n    for pr in range(len(mainSample.pt.table[key])):\n        build_spcr([mainSample.pt.table[key][pr][0]], md1)\n        build_spcr([mainSample.pt.table[key][pr][1]], md1)\n'

In [12]:
#     Генерация правил №2
# глубина базового перебора -- 2
# ограничение на длину правил -- 150
# минимальный уровень значимости Фишера -- 0.05
# минимальный порог увеличения вероятности правила - 0.0 

md2 = BaseModel(mainSample, base_depth=2, fully_depth=150, confidence_level = 0.05, negative_threshold=0.00,
               rules_write_path = 'exp_titanic/rules/base2/')

# Алгоритм поиска закономерностей: 
# итерация по всем позитивным предикатам в pt
# алгоритм принимает на вход предикат и генерирует все правила, предсказывающие этот предикат,
# а также все правила, предсказывающие отрицание этого предиката

# Поиск правил занял ~2.5 часа на 1 CPU

# для генерации правил раскомментировать следующий код:
"""
for key in mainSample.pt.table.keys():
    for pr in range(len(mainSample.pt.table[key])):
        build_spcr([mainSample.pt.table[key][pr][0]], md2)
        build_spcr([mainSample.pt.table[key][pr][1]], md2)
"""

'\nfor key in mainSample.pt.table.keys():\n    for pr in range(len(mainSample.pt.table[key])):\n        build_spcr([mainSample.pt.table[key][pr][0]], md2)\n        build_spcr([mainSample.pt.table[key][pr][1]], md2)\n'

In [13]:
#     Генерация правил №3
# глубина базового перебора -- 3
# ограничение на длину правил -- 150
# минимальный уровень значимости Фишера -- 0.05
# минимальный порог увеличения вероятности правила - 0.0 

md3 = BaseModel(mainSample, base_depth=3, fully_depth=150, confidence_level = 0.05, negative_threshold=0.00,
               rules_write_path = 'exp_titanic/rules/base3/')

# Алгоритм поиска закономерностей: 
# итерация по всем позитивным предикатам в pt
# алгоритм принимает на вход предикат и генерирует все правила, предсказывающие этот предикат,
# а также все правила, предсказывающие отрицание этого предиката

# Поиск правил занял меньше 1 часа на 62 CPU

# для генерации правил раскомментировать следующий код:
"""
for key in mainSample.pt.table.keys():
    for pr in range(len(mainSample.pt.table[key])):
        build_spcr([mainSample.pt.table[key][pr][0]], md3)
        build_spcr([mainSample.pt.table[key][pr][1]], md3)
"""

'\nfor key in mainSample.pt.table.keys():\n    for pr in range(len(mainSample.pt.table[key])):\n        build_spcr([mainSample.pt.table[key][pr][0]], md3)\n        build_spcr([mainSample.pt.table[key][pr][1]], md3)\n'

In [ ]:
#TODO: резюме поиска законов

## Моделирование нейронной сети

Этапы моделирования нейронной сети:
1. Используя сгенерированные вероятностные законы построить ассоциативные правила
2. Посчитать вероятности у найденных правил и оставить только те, которые удовлетворяют критериям
3. Сгенерировать новые предикаты, объединив в одну конъюнкцию посылку и заключение ассоцитивных правил
4. Преобразовать датасет, заменив в нём все признаки объектов на новые предикаты (все признаки будут булевы)
5. Добавить целеой признак labels для объектов датасета
6. Найти все вероятностные законы, предсказывающие целевой признак
7. Отобрать все правила, удовлетворяющие критериям
8. Используя тестовую выборку, получить предсказания по сгенерированным правилам

### Построение ассоциативных правил

In [14]:
from probconcepts.network.ext_model import ExtBaseModel
from probconcepts.network.process import *

In [15]:
# создание модели (необходима для вычисления вероятностей)
extModel = ExtBaseModel(mainSample)

# преобразованный словарь типов, необходим для считывания законов из файла
ctype_dict = {i : extModel.sample.cd.type_dict[j] for i, j in enumerate(extModel.sample.cd.type_dict)}

In [16]:
# загрузка правил с базовым перебором длины 3
extrules = find_extrules("exp_titanic/rules/base3/", extModel, ctype_dict)

Total rules: 32276
Total unique premises: 25285
Total extrules: 25285


In [17]:
# вычиссление вероятностей для расширенных правил
for extrule in tqdm(extrules):
    extrule.evaluate(extModel)

100%|██████████| 25285/25285 [01:26<00:00, 291.54it/s]


## ТЕСТ №1

In [98]:
# ищем подходящие правила и преобразуем их в конъюнкции. Это будут новые признаки

sort_rules = []
for extrule in extrules:
    if extrule.prob > 0.70 and len(extrule.conclusion) >= 3 and len(extrule.premise) >= 3:
        num_pos = 0
        for pr in extrule.premise:
            if pr.is_positive():
                num_pos += 1
        if num_pos >= 1:
            num_pos = 0
            for pr in extrule.conclusion:
                if pr.is_positive():
                    num_pos += 1
            if num_pos >= 1:
                sort_rules.append(extrule)
        
print("total sort rules:", len(sort_rules))
sort_conjs = [extrule.to_conjunction() for extrule in sort_rules]

train_lbls = pd.read_csv(PATH+"processed data/train_lbls_titanic.csv")
new_df = convert_sample(mainSample, sort_conjs)
new_df["Survived"] = train_lbls["Survived"]
print(new_df.head())

lbl = len(sort_conjs)

bools = list(range(0, len(sort_conjs)))
cats = [lbl]


NNsample1 = Sample(data=new_df, bool_features=bools, cat_features=cats, label=lbl,
                cd_output_path=ENC_PATH+"NNexttrain_cd.json", encoding_output_path=ENC_PATH+"NNexttrain_encode.json")

NN1ctype_dict = {i : NNsample1.cd.type_dict[j] for i, j in enumerate(NNsample1.cd.type_dict)}

print(NN1ctype_dict)
print("Label:", lbl)

total sort rules: 58
       0      1      2      3      4      5      6      7      8      9  ...  \
0  False  False  False  False  False  False   True  False  False  False  ...   
1  False  False  False  False  False  False  False  False  False  False  ...   
2  False  False  False  False  False  False  False  False  False  False  ...   
3  False  False  False  False  False  False  False  False  False  False  ...   
4  False  False  False  False  False  False  False  False  False  False  ...   

      49     50     51     52     53     54     55     56     57 Survived  
0   True  False  False   True  False  False  False  False  False        0  
1  False  False  False  False  False  False  False  False  False        1  
2  False  False  False  False   True  False  False  False  False        1  
3  False  False  False  False  False  False  False  False  False        1  
4  False   True   True   True  False   True  False   True   True        0  

[5 rows x 59 columns]
{0: 'B', 1: 'B', 2:

In [19]:
#     Моделирование NN #1
# глубина базового перебора -- 2
# ограничение на длину правил -- +inf
# минимальный уровень значимости Фишера -- 0.05
# минимальный порог увеличения вероятности правила - 0.00 

NNmd1 = BaseModel(NNsample1, base_depth=2, fully_depth=999, confidence_level = 0.05, negative_threshold=0.00,
               rules_write_path = 'exp_titanic/rules/T1NNRulesBase3_2/')

# Алгоритм поиска закономерностей: 
# итерация по всем позитивным предикатам в pt
# алгоритм принимает на вход предикат и генерирует все правила, предсказывающие этот предикат,
# а также все правила, предсказывающие отрицание этого предиката

# для генерации правил раскомментировать следующий код:
#build_spcr([NNsample.pt.table[lbl][0][0]], NNmd1)
#build_spcr([NNsample.pt.table[lbl][0][1]], NNmd1)

### ПРОВЕРКА ТОЧНОСТИ

In [99]:
neg_rules = decstr("exp_titanic/rules/T1NNRulesBase3_2/spcr_<#58 = 0>.txt", NN1ctype_dict)  # негативные правила (0)
pos_rules = decstr("exp_titanic/rules/T1NNRulesBase3_2/spcr_<#58 != 0>.txt", NN1ctype_dict) # позитивные правила (1)

# убираем все правила с lbl в посылке
neg_rules_sort = []
pos_rules_sort = []
for rule in tqdm(neg_rules):
    add = True
    for pr in rule.premise:
        if pr.ident == lbl:
            add = False
            break
    if add:
        neg_rules_sort.append(rule)

for rule in tqdm(pos_rules):
    add = True
    for pr in rule.premise:
        if pr.ident == lbl:
            add = False
            break
    if add:
        pos_rules_sort.append(rule)

neg_rules_sort = [rule for rule in neg_rules_sort if rule.pvalue < 1e-10] #rule.prob > 0.5]
pos_rules_sort = [rule for rule in pos_rules_sort if rule.pvalue < 1e-10] #rule.prob > 0.5]
print(len(neg_rules_sort))
print(len(pos_rules_sort))

num_pr = len(NN1ctype_dict)-1 - len(sort_conjs)
types = ["pr"]*num_pr + ["conj"]*len(sort_conjs)

print("Number common features:", num_pr)
print("Number conjs features:", len(sort_conjs))
print(types)

100%|██████████| 3677/3677 [00:00<00:00, 219530.49it/s]

357
3592
Number common features: 0
Number conjs features: 58
['conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj']


In [22]:
preds, sel_preds, metrics, sel_metrics = make_predictions(mainSample.data, y_train, neg_rules_sort, pos_rules_sort, types, sort_conjs,  LOGS_PATH+"NN1_TR_LOGS.txt", LOGS_PATH+"NN1_TR_RULES.txt")
print("All objects")
for key, value in metrics.items():
    print(key, ' : ', value)
print()
print("Selected objects")
for key, value in sel_metrics.items():
    print(key, ' : ', value)

100%|██████████| 891/891 [01:06<00:00, 13.30it/s]

All objects
accuracy  :  0.35353535353535354
precision  :  0.3645833333333333
recall  :  0.9210526315789473
f1  :  0.5223880597014926
roc_auc  :  0.4605263157894737

Selected objects


In [23]:
preds, sel_preds, metrics, sel_metrics = make_predictions(mainSampleTest.data, y_test, neg_rules_sort, pos_rules_sort, types, sort_conjs, LOGS_PATH+"NN1_T_LOGS.txt", LOGS_PATH+"NN1_T_RULES.txt")
print("All objects")
for key, value in metrics.items():
    print(key, ' : ', value)
print()
print("Selected objects")
for key, value in sel_metrics.items():
    print(key, ' : ', value)

100%|██████████| 418/418 [00:33<00:00, 12.55it/s]


All objects
accuracy  :  0.37799043062200954
precision  :  0.37799043062200954
recall  :  1.0
f1  :  0.548611111111111
roc_auc  :  0.5

Selected objects


In [100]:
preds, sel_preds, metrics, sel_metrics = make_predictions(mainSampleTest.data, y_test, neg_rules_sort, pos_rules_sort, types, sort_conjs, LOGS_PATH+"NN1_T_LOGS.txt", LOGS_PATH+"NN1_T_RULES.txt")
print("All objects")
for key, value in metrics.items():
    print(key, ' : ', value)
print()
print("Selected objects")
for key, value in sel_metrics.items():
    print(key, ' : ', value)

100%|██████████| 418/418 [00:35<00:00, 11.78it/s]

All objects
accuracy  :  0.6220095693779905
precision  :  0.5
recall  :  0.16455696202531644
f1  :  0.2476190476190476
roc_auc  :  0.5322784810126583

Selected objects


## ТЕСТ №2

добавление полученных конъюнкций к исходным признакам

In [90]:
sort_rules = []
for extrule in extrules:
    if extrule.prob > 0.90 and len(extrule.conclusion) >= 2 and len(extrule.premise) >= 2:
        num_pos = 0
        for pr in extrule.premise:
            if pr.is_positive():
                num_pos += 1
        if num_pos >= 2:
            num_pos = 0
            for pr in extrule.conclusion:
                if pr.is_positive():
                    num_pos += 1
            if num_pos >= 2:
                sort_rules.append(extrule)
print(len(sort_rules))
sort_conjs = [extrule.to_conjunction() for extrule in sort_rules]
train_lbls = pd.read_csv(PATH+"processed data/train_lbls_titanic.csv")
df = pd.read_csv(PATH + "processed data/train_titanic.csv")
new_df = convert_sample(mainSample, sort_conjs)
ext_df = pd.concat([df, new_df], axis=1)
ext_df["Survived"] = train_lbls["Survived"]
print(ext_df.head())

num_f = 7
lbl = num_f + len(sort_conjs)

bools = [1] + list(range(num_f, num_f+len(sort_conjs)))
cats = [0,2,3,4,5,6] + [lbl]

NNsample2 = Sample(data=ext_df, bool_features=bools, cat_features=cats, label=lbl,
                cd_output_path=ENC_PATH+"NN2train_cd.json", encoding_output_path=ENC_PATH+"NN2train_encode.json")

NN2ctype_dict = {i : NNsample2.cd.type_dict[j] for i, j in enumerate(NNsample2.cd.type_dict)}

print(NN2ctype_dict)
print("Label:", lbl)

23
   Pclass     Sex Embarked Title  Age_cat Fare_cat  FSize      0      1  \
0       3    male        S    Mr  Student  quart_1      2  False  False   
1       1  female        C   Mrs    Adult  quart_4      2  False  False   
2       3  female        S  Miss    Young  quart_1      1  False  False   
3       1  female        S   Mrs    Young  quart_4      2  False  False   
4       3    male        S    Mr    Young  quart_2      1  False  False   

       2  ...     14     15     16     17     18     19     20     21     22  \
0  False  ...  False  False  False  False  False   True  False  False  False   
1  False  ...  False  False  False  False  False  False  False  False  False   
2  False  ...  False  False  False  False  False  False  False  False  False   
3  False  ...  False  False  False  False  False  False  False  False  False   
4  False  ...   True  False  False  False  False  False  False  False  False   

  Survived  
0        0  
1        1  
2        1  
3        1  


In [84]:
#     Моделирование NN #2
# глубина базового перебора -- 2
# ограничение на длину правил -- +inf
# минимальный уровень значимости Фишера -- 0.05
# минимальный порог увеличения вероятности правила - 0.00 

NNmd2 = BaseModel(NNsample2, base_depth=2, fully_depth=999, confidence_level = 0.05, negative_threshold=0.00,
               rules_write_path = 'exp_titanic/rules/T2NNRulesBase3_2/')

# Алгоритм поиска закономерностей: 
# итерация по всем позитивным предикатам в pt
# алгоритм принимает на вход предикат и генерирует все правила, предсказывающие этот предикат,
# а также все правила, предсказывающие отрицание этого предиката

# для генерации правил раскомментировать следующий код:
#build_spcr([NNsample2.pt.table[lbl][0][0]], NNmd2)
#build_spcr([NNsample2.pt.table[lbl][0][1]], NNmd2)

### проверка точности

In [91]:
neg_rules = decstr("exp_titanic/rules/T2NNRulesBase3_2/spcr_<#30 = 0>.txt", NN2ctype_dict)  # негативные правила (0)
pos_rules = decstr("exp_titanic/rules/T2NNRulesBase3_2/spcr_<#30 != 0>.txt", NN2ctype_dict) # позитивные правила (1)
lbl = 30
# убираем все правила с lbl в посылке

neg_rules_sort = []
pos_rules_sort = []
for rule in tqdm(neg_rules):
    add = True
    for pr in rule.premise:
        if pr.ident == lbl:
            add = False
            break
    if add:
        neg_rules_sort.append(rule)

for rule in tqdm(pos_rules):
    add = True
    for pr in rule.premise:
        if pr.ident == lbl:
            add = False
            break
    if add:
        pos_rules_sort.append(rule)

neg_rules_sort = [rule for rule in neg_rules_sort if rule.pvalue < 1e-55]
pos_rules_sort = [rule for rule in pos_rules_sort if rule.pvalue < 1e-55]

num_pr = len(NN2ctype_dict)-1 - len(sort_conjs)
types = ["pr"]*num_pr + ["conj"]*len(sort_conjs)

y_train = train_lbls.values.flatten()

print("Number common features:", num_pr)
print("Number conjs features:", len(sort_conjs))
print(types)
print(len(sort_conjs)+len(cats)+len(bools) == len(types)+1)

print(len(neg_rules_sort))
print(len(pos_rules_sort))


100%|██████████| 2885/2885 [00:00<00:00, 245806.59it/s]

Number common features: 7
Number conjs features: 23
['pr', 'pr', 'pr', 'pr', 'pr', 'pr', 'pr', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj']
False
8
25


In [67]:
# i = 0
# print(sort_conjs[i])
# print()
# print(str_conj_rule(neg_rules_sort[i], NNsample2, types, sort_conjs))
# print(NNsample2.pe.inverse_transform(neg_rules_sort[0].premise[4].operation.params))   # neg_rules_sort[0].premise[0])

In [92]:

preds, sel_preds, metrics, sel_metrics = make_predictions(mainSample.data, y_train, neg_rules_sort, pos_rules_sort, types, sort_conjs, LOGS_PATH+"NN2_TR_LOGS.txt", LOGS_PATH+"NN2_TR_RULES.txt")
print("All objects")
for key, value in metrics.items():
    print(key, ' : ', value)
print()
print("Selected objects")
for key, value in sel_metrics.items():
    print(key, ' : ', value)

100%|██████████| 891/891 [00:00<00:00, 2685.87it/s]

All objects
accuracy  :  0.8002244668911336
precision  :  0.7515337423312883
recall  :  0.716374269005848
f1  :  0.7335329341317366
roc_auc  :  0.7844166426996453

Selected objects


In [93]:
preds, sel_preds, metrics, sel_metrics = make_predictions(mainSampleTest.data, y_test, neg_rules_sort, pos_rules_sort, types, sort_conjs, LOGS_PATH+"NN2_T_LOGS.txt", LOGS_PATH+"NN2_T_RULES.txt")
print("All objects")
for key, value in metrics.items():
    print(key, ' : ', value)
print()
print("Selected objects")
for key, value in sel_metrics.items():
    print(key, ' : ', value)

100%|██████████| 418/418 [00:00<00:00, 2099.54it/s]

All objects
accuracy  :  0.7727272727272727
precision  :  0.7032258064516129
recall  :  0.689873417721519
f1  :  0.6964856230031949
roc_auc  :  0.7564751703992211

Selected objects


In [ ]:
All objects
accuracy  :  0.7655502392344498
precision  :  0.6973684210526315
recall  :  0.6708860759493671
f1  :  0.6838709677419355
roc_auc  :  0.7469814995131451

In [93]:
#rules_neg = make_predictions(mainSampleTest.data, y_test, neg_rules_sort, pos_rules_sort, types, sort_conjs, "0objLOGS.txt", "0objRULES.txt")
str(rules_neg[0])

'<#0 != 1> & <#3 = 0> & <#24 = False> => <#30 = 0> 0.9030612244897959, 7.450641437326582e-61'

In [ ]:
# i = 0
# print(sort_conjs[i])
# print()
# print(str_conj_rule(neg_rules_sort[i], NNsample2, types, sort_conjs))
# print(NNsample2.pe.inverse_transform(neg_rules_sort[0].premise[4].operation.params))   # neg_rules_sort[0].premise[0])

In [89]:
print(sort_conjs[24-7])

<#0 = 3> & <#5 != 0> & <#5 != 2> & <#6 = 1> & <#3 = 0> & <#1 = False>


In [94]:
print(str_conj_rule(rules_neg[0], NNsample2, types, sort_conjs))

<#Pclass != 1> & <#Title = Mr> & <(<#Pclass = 3> & <#Fare_cat != quart_1> & <#Fare_cat != quart_2> & <#FSize = 1> & <#Title = Mr> & <#Sex = male>) = False> => <#Survived = 0> 0.9030612244897959 7.450641437326582e-61


In [ ]:
# |0:3| |1:False| |2:0| |3:0| |4:0| |5:0| |6:1|

## ТЕСТ №3

предсказывание на исходном датасете

In [94]:
conjs = []
train_lbls = pd.read_csv(PATH+"processed data/train_lbls_titanic.csv")
new_df = df
new_df["Survived"] = train_lbls["Survived"]
print(new_df.head())

num_f = 7
lbl = num_f

bools = [1]
cats = [0,2,3,4,5,6] + [lbl]


NNsample3 = Sample(data=new_df, bool_features=bools, cat_features=cats, label=lbl,
                cd_output_path=ENC_PATH+"NNexttrain_cd.json", encoding_output_path=ENC_PATH+"NNexttrain_encode.json")

NN3ctype_dict = {i : NNsample3.cd.type_dict[j] for i, j in enumerate(NNsample3.cd.type_dict)}

print(NN3ctype_dict)
print("Label:", lbl)

   Pclass     Sex Embarked Title  Age_cat Fare_cat  FSize  Survived
0       3    male        S    Mr  Student  quart_1      2         0
1       1  female        C   Mrs    Adult  quart_4      2         1
2       3  female        S  Miss    Young  quart_1      1         1
3       1  female        S   Mrs    Young  quart_4      2         1
4       3    male        S    Mr    Young  quart_2      1         0
{0: 'C', 1: 'B', 2: 'C', 3: 'C', 4: 'C', 5: 'C', 6: 'C', 7: 'C'}
Label: 7


In [71]:
#     Моделирование NN #3
# глубина базового перебора -- 2
# ограничение на длину правил -- +inf
# минимальный уровень значимости Фишера -- 0.05
# минимальный порог увеличения вероятности правила - 0.00 

NNmd3 = BaseModel(NNsample3, base_depth=2, fully_depth=999, confidence_level = 0.05, negative_threshold=0.00,
               rules_write_path = 'exp_titanic/rules/T3NNRulesBase3_2/', )

# Алгоритм поиска закономерностей: 
# итерация по всем позитивным предикатам в pt
# алгоритм принимает на вход предикат и генерирует все правила, предсказывающие этот предикат,
# а также все правила, предсказывающие отрицание этого предиката

# для генерации правил раскомментировать следующий код:
#build_spcr([NNsample3.pt.table[lbl][0][0]], NNmd3)
#build_spcr([NNsample3.pt.table[lbl][0][1]], NNmd3)

### ПРОВЕРКА ТОЧНОСТИ

In [95]:
neg_rules = decstr("exp_titanic/rules/T3NNRulesBase3_2/spcr_<#7 = 0>.txt", NN3ctype_dict)  # негативные правила (0)
pos_rules = decstr("exp_titanic/rules/T3NNRulesBase3_2/spcr_<#7 != 0>.txt", NN3ctype_dict) # позитивные правила (1)

# убираем все правила с lbl в посылке
neg_rules_sort = []
pos_rules_sort = []
for rule in tqdm(neg_rules):
    add = True
    for pr in rule.premise:
        if pr.ident == lbl:
            add = False
            break
    if add:
        neg_rules_sort.append(rule)

for rule in tqdm(pos_rules):
    add = True
    for pr in rule.premise:
        if pr.ident == lbl:
            add = False
            break
    if add:
        pos_rules_sort.append(rule)

neg_rules_sort = [rule for rule in neg_rules_sort if rule.pvalue < 1e-50]
pos_rules_sort = [rule for rule in pos_rules_sort if rule.pvalue < 1e-50]
print(len(neg_rules_sort))
print(len(pos_rules_sort))

num_pr = len(NN3ctype_dict)-1 - len(conjs)
types = ["pr"]*num_pr + ["conj"]*len(conjs)

y_train = train_lbls.values.flatten()

print("Number common features:", num_pr)
print("Number conjs features:", len(conjs))
print(types)

100%|██████████| 197/197 [00:00<00:00, 180607.19it/s]

11
33
Number common features: 7
Number conjs features: 0
['pr', 'pr', 'pr', 'pr', 'pr', 'pr', 'pr']


In [96]:
preds, sel_preds, metrics, sel_metrics = make_predictions(mainSample.data, y_train, neg_rules_sort, pos_rules_sort, types, [], LOGS_PATH+"NN3_TR_LOGS.txt", LOGS_PATH+"NN3_TR_RULES.txt")

print("All objects")
for key, value in metrics.items():
    print(key, ' : ', value)
print()
print("Selected objects")
for key, value in sel_metrics.items():
    print(key, ' : ', value)

100%|██████████| 891/891 [00:00<00:00, 2418.14it/s]

All objects
accuracy  :  0.8013468013468014
precision  :  0.7522935779816514
recall  :  0.7192982456140351
f1  :  0.73542600896861
roc_auc  :  0.7858786310037389

Selected objects


In [97]:
# проверка на тестовой выборке
preds, sel_preds, metrics, sel_metrics = make_predictions(mainSampleTest.data, y_test, neg_rules_sort, pos_rules_sort, types, [], LOGS_PATH+"NN3_T_LOGS.txt", LOGS_PATH+"NN3_T_RULES.txt")
print("All objects")
for key, value in metrics.items():
    print(key, ' : ', value)
print()
print("Selected objects")
for key, value in sel_metrics.items():
    print(key, ' : ', value)

100%|██████████| 418/418 [00:00<00:00, 2038.73it/s]

All objects
accuracy  :  0.7655502392344498
precision  :  0.6973684210526315
recall  :  0.6708860759493671
f1  :  0.6838709677419355
roc_auc  :  0.7469814995131451

Selected objects


## Вторая часть тестов

Добавление конъюнкций в признаки не повысило точности предсказаний, а предсказания на исходых признаков были лучше, чем  предсказания с использованием конъюнкций. В этой части тестов конъюнкции будут выбираться не просто по вероятностям, а по количеству объектов, на которых они выполняются

In [101]:
from probconcepts.network.ext_mesure import calc_supp

In [102]:
# предполагается, что правила уже загружены и вероятности в них посчитаны

# хеширование правила по посылке

def hash_by_premise(extrule):
    prem_set = set(extrule.premise)
    return hash(frozenset(prem_set))

def hash_by_concl(extrule):
    concl_set = set(extrule.conclusion)
    return hash(frozenset(concl_set))

# правила с уникальной посылкой и заключением
same_concl_rules = {}

for extrule in extrules:
    try:
        same_concl_rules[hash_by_concl(extrule)].append(extrule)
    except:
        same_concl_rules[hash_by_concl(extrule)] = list()
        same_concl_rules[hash_by_concl(extrule)].append(extrule)
        
# отбор правил с носителем >100 и вероятностью на носителе > 0.5
to_MSR_rules = []
for k in tqdm(same_concl_rules.keys()):
    arr = []
    for rule in same_concl_rules[k]:
        top, bot = calc_supp(rule, extModel)
        if rule.prob > 0.5 and bot > 100:
            arr.append(rule)
    to_MSR_rules.append(arr)
    
len(to_MSR_rules)    

100%|██████████| 1286/1286 [01:26<00:00, 14.81it/s]


1286

In [103]:
i = 0
for pr in range(len(to_MSR_rules)):
    i += len(to_MSR_rules[pr])
print(i)        

14949


In [104]:
MSR_rules = []
for k in range(len(to_MSR_rules)):
    try: 
        max_prob_rule = to_MSR_rules[k][0]
        for rule in to_MSR_rules[k]:
            if rule.prob > max_prob_rule.prob:
                max_prob_rule = rule
        MSR_rules.append(max_prob_rule)
    except:
        continue

print("Total MSR rules:", len(MSR_rules))

Total MSR rules: 250


In [79]:
for rule in MSR_rules: print(rule)

<#0 != 1> & <#4 = 0> & <#6 != 5> & <#6 != 3> & <#6 != 4> => <#5 != 1>  0.9833333333333333, 3.014899689546847e-17
<#1 = False> & <#4 != 3> & <#6 != 2> & <#6 != 3> & <#6 != 6> & <#6 != 4> & <#6 != 8> => <#5 != 1> & <#6 = 1>  0.9039735099337748, 1.7052650009978737e-89
<#0 = 3> & <#4 != 2> & <#6 = 1> => <#5 != 1> & <#4 = 0>  0.5533333333333333, 1.0568800325190567e-33
<#0 = 2> & <#4 != 3> & <#6 != 3> => <#5 != 1> & <#5 = 2>  0.6, 7.239240198853809e-21
<#0 = 3> & <#1 = False> & <#4 != 3> & <#6 != 6> => <#5 != 1> & <#5 = 0>  0.5454545454545454, 1.4125336951860918e-46
<#0 = 3> & <#1 = False> & <#6 != 6> & <#6 != 8> => <#5 != 1> & <#3 = 0>  0.9118541033434651, 3.5745763910810985e-100
<#0 != 1> & <#0 != 2> & <#2 != 0> => <#5 != 1> & <#2 = 2>  0.5214285714285715, 2.190366183052352e-62
<#0 != 1> & <#3 != 2> & <#6 != 3> => <#5 != 1> & <#6 = 1> & <#3 = 0> & <#1 = False>  0.6652452025586354, 3.2514179325418264e-73
<#1 = False> & <#4 != 1> & <#4 != 3> & <#6 != 6> & <#6 != 8> => <#5 != 1> & <#6 = 1> & 

In [105]:
msr_conjs = [extrule.to_conjunction() for extrule in MSR_rules]
msr_df = convert_sample(mainSample, msr_conjs)
msr_df

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,True,False,False,False,True,True,False,False,False,False,...,False,False,False,True,False,True,False,True,True,True
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,True,False,True
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,True
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,True,False,True,False,True
4,False,True,False,False,False,True,False,True,True,False,...,False,True,True,False,False,True,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,False,True,False,True,False,False,False,False,False,True,...,False,True,False,False,False,True,False,True,False,True
887,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,True
888,None,False,None,False,False,False,False,False,False,False,...,True,False,False,False,False,True,False,None,None,None
889,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,True,False,True,True,False,True


In [107]:
sort_MSR = []
for rule in MSR_rules:
    if rule.prob > 0.95:
        sort_MSR.append(rule)
        print(rule)
        print(calc_supp(rule, extModel))
len(sort_MSR)

<#0 != 1> & <#4 = 0> & <#6 != 5> & <#6 != 3> & <#6 != 4> => <#5 != 1>  0.9833333333333333, 3.014899689546847e-17
(117, 118)
<#1 = True> & <#3 != 2> & <#3 != 4> & <#3 != 5> => <#3 = 1>  0.9922480620155039, 1.02338702237422e-157
(127, 127)
<#3 = 1> => <#3 != 0> & <#4 != 5> & <#2 != 2> & <#3 != 2> & <#3 != 3> & <#1 = True> & <#4 != 3>  0.972972972972973, 1.3383994696408653e-122
(107, 109)
<#3 = 2> => <#3 != 0> & <#3 != 1> & <#3 != 5> & <#4 != 6> & <#3 != 3> & <#1 = True>  0.9866666666666667, 1.6201139098002705e-152
(147, 148)
<#1 = True> => <#3 != 0> & <#3 != 5> & <#4 != 6> & <#3 != 3>  0.9809160305343512, 1.148086393850135e-186
(256, 260)
<#0 = 1> & <#3 != 3> => <#4 != 5>  0.9945652173913043, 0.000544201617535986
(182, 182)
<#3 = 0> => <#4 != 5> & <#3 != 1> & <#3 != 5> & <#6 != 5> & <#3 != 4> & <#6 != 4> & <#3 != 2> & <#3 != 3> & <#1 = False> & <#4 != 3> & <#6 != 7>  0.9775, 2.6327306067019033e-196
(390, 398)
<#3 != 2> & <#3 != 3> => <#4 != 5> & <#6 != 5> & <#4 != 3>  0.9887005649717514,

63

In [108]:
# посчитаем кол-во объектов, на которых выполнимы конъюнкции, 
# а также кол-во конъюнкций, выполнимых на каждом объекте
conj_perf = [0]*len(msr_conjs)
obj_perf = []
for i in range(len(msr_df)):
    true_on_obj = 0
    obj = msr_df.iloc[i]
    for i in obj.keys():
        if obj[i] == True:
            true_on_obj += 1
            conj_perf[i] += 1
    obj_perf.append(true_on_obj)

In [ ]:
# попробуем взять конъюнкции из sort_MSR (63 правила)

## ТЕСТ 5

только конъюнкции

In [109]:
conjs5 = [extrule.to_conjunction() for extrule in sort_MSR]

train_lbls = pd.read_csv(PATH+"processed data/train_lbls_titanic.csv")
df5 = convert_sample(mainSample, conjs5)
df5["Survived"] = train_lbls["Survived"]
print(df5.head())

lbl = len(conjs5)

bools = list(range(0, len(conjs5)))
cats = [lbl]


NNsample5 = Sample(data=df5, bool_features=bools, cat_features=cats, label=lbl,
                cd_output_path=ENC_PATH+"NN5train_cd.json", encoding_output_path=ENC_PATH+"NN5train_encode.json")

NN5ctype_dict = {i : NNsample5.cd.type_dict[j] for i, j in enumerate(NNsample5.cd.type_dict)}

print(NN5ctype_dict)
print("Label:", lbl)

       0      1      2      3      4      5      6      7      8      9  ...  \
0   True  False  False  False  False  False   True   True   True  False  ...   
1  False   True   True  False   True   True  False   True  False  False  ...   
2  False  False  False   True   True  False  False  False  False   True  ...   
3  False   True   True  False   True   True  False   True  False   True  ...   
4  False  False  False  False  False  False   True   True  False   True  ...   

      54     55     56     57     58     59     60    61    62 Survived  
0  False  False  False  False  False   True  False  True  True        0  
1   True   True   True  False  False  False   True  True  True        1  
2  False  False  False  False  False   True  False  True  True        1  
3   True   True   True  False  False   True  False  True  True        1  
4  False  False  False   True  False   True  False  True  True        0  

[5 rows x 64 columns]
{0: 'B', 1: 'B', 2: 'B', 3: 'B', 4: 'B', 5: 'B', 6: 

In [110]:
#     Моделирование NN #5

NNmd5 = BaseModel(NNsample5, base_depth=2, fully_depth=999, confidence_level = 0.05, negative_threshold=0.00,
               rules_write_path = 'exp_titanic/rules/T5NNRulesBase3_2/', )
# 1 час 1 CPU
# для генерации правил раскомментировать следующий код:
#build_spcr([NNsample5.pt.table[lbl][0][0]], NNmd5)
#build_spcr([NNsample5.pt.table[lbl][0][1]], NNmd5)

In [117]:
neg_rules = decstr("exp_titanic/rules/T5NNRulesBase3_2/spcr_<#63 = 0>.txt", NN5ctype_dict)  # негативные правила (0)
pos_rules = decstr("exp_titanic/rules/T5NNRulesBase3_2/spcr_<#63 != 0>.txt", NN5ctype_dict) # позитивные правила (1)

# убираем все правила с lbl в посылке
neg_rules_sort = []
pos_rules_sort = []
for rule in tqdm(neg_rules):
    add = True
    for pr in rule.premise:
        if pr.ident == lbl:
            add = False
            break
    if add:
        neg_rules_sort.append(rule)

for rule in tqdm(pos_rules):
    add = True
    for pr in rule.premise:
        if pr.ident == lbl:
            add = False
            break
    if add:
        pos_rules_sort.append(rule)

neg_rules_sort = [rule for rule in neg_rules_sort if rule.pvalue < 1e-30]#rule.prob > 0.65]
pos_rules_sort = [rule for rule in pos_rules_sort if rule.pvalue < 1e-30]#rule.prob > 0.65]
print(len(neg_rules_sort))
print(len(pos_rules_sort))

num_pr = len(NN5ctype_dict)-1 - len(conjs5)
types = ["pr"]*num_pr + ["conj"]*len(conjs5)

print("Number common features:", num_pr)
print("Number conjs features:", len(conjs5))
print(types)

100%|██████████| 2295/2295 [00:00<00:00, 526899.54it/s]

907
474
Number common features: 0
Number conjs features: 63
['conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj']


In [113]:
preds, sel_preds, metrics, sel_metrics = make_predictions(mainSample.data, y_train, neg_rules_sort, pos_rules_sort, types, conjs5, LOGS_PATH+"NN5_TR_LOGS.txt", LOGS_PATH+"NN5_TR_RULES.txt")
print("All objects")
for key, value in metrics.items():
    print(key, ' : ', value)
print()
print("Selected objects")
for key, value in sel_metrics.items():
    print(key, ' : ', value)

100%|██████████| 891/891 [00:00<00:00, 2303.97it/s]


All objects
accuracy  :  0.5117845117845118
precision  :  0.3822784810126582
recall  :  0.4415204678362573
f1  :  0.4097693351424695
roc_auc  :  0.4985380116959065

Selected objects


In [118]:
preds, sel_preds, metrics, sel_metrics = make_predictions(mainSampleTest.data, y_test, neg_rules_sort, pos_rules_sort, types, conjs5, LOGS_PATH+"NN5_T5_LOGS.txt", LOGS_PATH+"NN5_T5_RULES.txt")
print("All objects")
for key, value in metrics.items():
    print(key, ' : ', value)
print()
print("Selected objects")
for key, value in sel_metrics.items():
    print(key, ' : ', value)

100%|██████████| 418/418 [00:07<00:00, 57.95it/s]


All objects
accuracy  :  0.6578947368421053
precision  :  0.5380710659898477
recall  :  0.6708860759493671
f1  :  0.5971830985915493
roc_auc  :  0.6604430379746835

Selected objects


## ТЕСТ 6

добавление конъюнкций к исходным признакам

In [119]:
conjs6 = [extrule.to_conjunction() for extrule in sort_MSR]
df = pd.read_csv(PATH + "processed data/train_titanic.csv")
train_lbls = pd.read_csv(PATH+"processed data/train_lbls_titanic.csv")
new_df = convert_sample(mainSample, conjs6)
df6 = pd.concat([df, new_df], axis=1)
df6["Survived"] = train_lbls["Survived"]
print(df6.head())

num_f = 7
lbl = num_f + len(conjs6)

bools = [1] + list(range(num_f, num_f+len(conjs6)))
cats = [0,2,3,4,5,6] + [lbl]

NNsample6 = Sample(data=df6, bool_features=bools, cat_features=cats, label=lbl,
                cd_output_path=ENC_PATH+"NN6train_cd.json", encoding_output_path=ENC_PATH+"NN6train_encode.json")

NN6ctype_dict = {i : NNsample6.cd.type_dict[j] for i, j in enumerate(NNsample6.cd.type_dict)}

print(NN6ctype_dict)
print("Label:", lbl)

   Pclass     Sex Embarked Title  Age_cat Fare_cat  FSize      0      1  \
0       3    male        S    Mr  Student  quart_1      2   True  False   
1       1  female        C   Mrs    Adult  quart_4      2  False   True   
2       3  female        S  Miss    Young  quart_1      1  False  False   
3       1  female        S   Mrs    Young  quart_4      2  False   True   
4       3    male        S    Mr    Young  quart_2      1  False  False   

       2  ...     54     55     56     57     58     59     60    61    62  \
0  False  ...  False  False  False  False  False   True  False  True  True   
1   True  ...   True   True   True  False  False  False   True  True  True   
2  False  ...  False  False  False  False  False   True  False  True  True   
3   True  ...   True   True   True  False  False   True  False  True  True   
4  False  ...  False  False  False   True  False   True  False  True  True   

  Survived  
0        0  
1        1  
2        1  
3        1  
4        0  

[

In [120]:
#     Моделирование NN #6

NNmd6 = BaseModel(NNsample6, base_depth=2, fully_depth=999, confidence_level = 0.05, negative_threshold=0.00,
               rules_write_path = 'exp_titanic/rules/T6NNRulesBase3_2/', )
# 5 часов 1 CPU
# для генерации правил раскомментировать следующий код:
#build_spcr([NNsample6.pt.table[lbl][0][0]], NNmd6)
#build_spcr([NNsample6.pt.table[lbl][0][1]], NNmd6)

In [121]:
neg_rules = decstr("exp_titanic/rules/T6NNRulesBase3_2/spcr_<#70 = 0>.txt", NN6ctype_dict)  # негативные правила (0)
pos_rules = decstr("exp_titanic/rules/T6NNRulesBase3_2/spcr_<#70 != 0>.txt", NN6ctype_dict) # позитивные правила (1)

# убираем все правила с lbl в посылке
neg_rules_sort = []
pos_rules_sort = []
for rule in tqdm(neg_rules):
    add = True
    for pr in rule.premise:
        if pr.ident == lbl:
            add = False
            break
    if add:
        neg_rules_sort.append(rule)

for rule in tqdm(pos_rules):
    add = True
    for pr in rule.premise:
        if pr.ident == lbl:
            add = False
            break
    if add:
        pos_rules_sort.append(rule)

neg_rules_sort = [rule for rule in neg_rules_sort if rule.pvalue < 1e-50] # rule.prob > 0.70]
pos_rules_sort = [rule for rule in pos_rules_sort if rule.pvalue < 1e-50] #rule.prob > 0.70]
print(len(neg_rules_sort))
print(len(pos_rules_sort))

num_pr = len(NN6ctype_dict)-1 - len(conjs6)
types = ["pr"]*num_pr + ["conj"]*len(conjs6)

y_train = train_lbls.values.flatten()

print("Number common features:", num_pr)
print("Number conjs features:", len(conjs6))
print(types)

100%|██████████| 9659/9659 [00:00<00:00, 374248.57it/s]

612
525
Number common features: 7
Number conjs features: 63
['pr', 'pr', 'pr', 'pr', 'pr', 'pr', 'pr', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj', 'conj']


In [212]:
preds, sel_preds, metrics, sel_metrics = make_predictions(mainSample.data, y_train, neg_rules_sort, pos_rules_sort, types, conjs6, LOGS_PATH+"NN6_TR_LOGS.txt", LOGS_PATH+"NN6_TR_RULES.txt")
print("All objects")
for key, value in metrics.items():
    print(key, ' : ', value)
print()
print("Selected objects")
for key, value in sel_metrics.items():
    print(key, ' : ', value)

100%|██████████| 891/891 [00:11<00:00, 78.96it/s]


All objects
accuracy  :  0.797979797979798
precision  :  0.7154255319148937
recall  :  0.7865497076023392
f1  :  0.7493036211699164
roc_auc  :  0.7958249448758509

Selected objects
accuracy  :  0.8138686131386861
precision  :  0.7421383647798742
recall  :  0.7687296416938111
f1  :  0.7552
roc_auc  :  0.8047531703614685


In [122]:
preds, sel_preds, metrics, sel_metrics = make_predictions(mainSampleTest.data, y_test, neg_rules_sort, pos_rules_sort, types, conjs6, LOGS_PATH+"NN6_TR_LOGS.txt", LOGS_PATH+"NN6_TR_RULES.txt")
print("All objects")
for key, value in metrics.items():
    print(key, ' : ', value)
print()
print("Selected objects")
for key, value in sel_metrics.items():
    print(key, ' : ', value)

100%|██████████| 418/418 [00:06<00:00, 59.93it/s]

All objects
accuracy  :  0.7607655502392344
precision  :  0.6726190476190477
recall  :  0.7151898734177216
f1  :  0.6932515337423314
roc_auc  :  0.75182570593963

Selected objects


In [ ]:


    
"""   
    
l_same_concl_rules = []
for k in same_concl_rules.keys():
    l_same_concl_rules.append(same_concl_rules[k])
    
max_spec_rules = []
for arr_by_concl in list_unique_rules:
    max_prob_rule = arr_by_concl[0]
    for rule in arr_by_concl:
        if rule.prob > max_prob_rule.prob:
            max_prob_rule = rule
    max_spec_rules.append(max_prob_rule)
print(len(max_spec_rules))
"""

threshold_rules = [] 
for rule in MSR_rules:
    if rule.prob > 0.4:
        threshold_rules.append(rule)
        
print(len(threshold_rules))

conjs = [extrule.to_conjunction() for extrule in threshold_rules]

#train_lbls = pd.read_csv(PATH+"processed data/train_lbls_titanic.csv")
new_df = convert_sample(mainSample, conjs)
new_df

conj_perf = [0]*len(conjs)
obj_perf = []
for i in range(len(new_df)):
    num_true_obj = 0
    obj = new_df.iloc[i]
    for i in obj.keys():
        if obj[i] == True:
            num_true_obj += 1
            conj_perf[i] += 1
    obj_perf.append(num_true_obj)
n = 0
high_perf_rules = []
for i, perf in enumerate(conj_perf):
    if perf > 300:
        high_perf_rules.append(threshold_rules[i])
        n+=1
    
    #print(i, perf)
print(n)

# 53
conjs = [extrule.to_conjunction() for extrule in high_perf_rules]

#train_lbls = pd.read_csv(PATH+"processed data/train_lbls_titanic.csv")
high_perf_df = convert_sample(mainSample, conjs)
high_perf_df


conj_perf = [0]*len(conjs)
obj_perf = []
for i in range(len(high_perf_df)):
    num_true_obj = 0
    obj = high_perf_df.iloc[i]
    for i in obj.keys():
        if obj[i] == True:
            num_true_obj += 1
            conj_perf[i] += 1
    obj_perf.append(num_true_obj)
    
za = 0
for obj in obj_perf:
    if obj > :
        za += 1
print(za)

for i, perf in enumerate(obj_perf):
    print(i, perf)
    
bools = list(range(len(conjs)))

TestSample = Sample(data=new_df, bool_features=bools,
                cd_output_path=PATH+"TESTtrain_cd.json", encoding_output_path=PATH+"TESTtrain_encode.json")

print(threshold_rules[446])

calc_probs(threshold_rules[446], extModel)

In [230]:
# to kaggle submission
survived = pd.DataFrame([preds, list(range(892,1310))]).T
survived.columns = ['Survived', 'PassengerId']#.reset_index()#to_csv('predicted.csv')
survived.to_csv('predicted.csv', index=False)